In [1]:
import numpy as np
import os

In [84]:
from pyloric import simulate, create_prior, summary_stats
import matplotlib.pyplot as plt
from sbi.inference import NPE
from copy import deepcopy
import pickle
import torch
from sbi.utils.user_input_checks_utils import PytorchReturnTypeWrapper

In [92]:
prior = create_prior(as_torch_dist=True)
prior = PytorchReturnTypeWrapper(prior)
parameter_set = prior.sample((1,))

In [99]:
with open("../../../Downloads/all_circuit_parameters.pkl", "rb") as handle:
    theta = pickle.load(handle)
with open("../../../Downloads/all_simulation_outputs.pkl", "rb") as handle:
    x = pickle.load(handle)
seeds = np.load("../../../Downloads/all_seeds.npy")
x_o = np.load(
    "../../../Downloads/xo_11deg.npy",
    allow_pickle=True,
)

clean_x = []
for i in range(18):
    x_ = x.to_numpy()[:, i]
    nan_vals = np.isnan(x_)
    if i == 0:
        x_[x_ > 3000] = 3000
        x_[nan_vals] = 3000

    if i in [1, 2, 3]:
        x_[x_ > 500] = 500
        x_[nan_vals] = 500

    if i in [4, 5, 6]:
        x_[x_ > 1] = 1
        x_[nan_vals] = 1

    if i in [7, 8]:
        x_[nan_vals] = 1.0

    if i in [9, 10, 11, 12]:
        x_[x_ < -1000] = -1000
    if i in [9, 10, 11, 12]:
        x_[x_ > 1000] = 1000
        x_[nan_vals] = 1000

    if i in [13, 14]:
        x_[x_ > 1] = 1
        x_[nan_vals] = 1
    if i in [13, 14]:
        x_[x_ < -1] = -1

    if i in [15, 16, 17]:
        x_[x_ > 10] = 10
        x_[nan_vals] = 100
        
    clean_x.append(x_)
#     fig, ax = plt.subplots(1, 1, figsize=(6, 3))
#     _ = ax.hist(x_, bins=100)
#     _ = ax.axvline(x_o[i], color="r")
#     plt.show()
#     print(np.nanmean(x.to_numpy()[:, i]), np.nanmedian(x.to_numpy()[:, i]), np.nanstd(x.to_numpy()[:, i]))

clean_x = torch.as_tensor(np.asarray(clean_x).T, dtype=torch.float32)
clean_theta = torch.as_tensor(theta.to_numpy(), dtype=torch.float32)

/var/folders/kv/39t9blx569l5_sxxzx89vb3c0000gn/T/ipykernel_26613/2642326754.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  theta = pickle.load(handle)
/var/folders/kv/39t9blx569l5_sxxzx89vb3c0000gn/T/ipykernel_26613/2642326754.py:4: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world us

In [100]:
# inference = NPE(prior, "nsf")
# inference = inference.append_simulations(clean_theta, clean_x)

In [168]:
density_estimator = inference.train(max_num_epochs=50, resume_training=True)

 Training neural network. Epochs trained: 51

In [169]:
with open("inference.pkl", "wb") as handle:
    pickle.dump(inference, handle)

/var/folders/kv/39t9blx569l5_sxxzx89vb3c0000gn/T/ipykernel_26613/1718262224.py:2: UserWarning: When the inference object is pickled, the behaviour of the loaded object changes in the following two ways: 1) `.train(..., retrain_from_scratch=True)` is not supported. 2) When the loaded object calls the `.train()` method, it generates a new tensorboard summary writer (instead of appending to the current one).
  pickle.dump(inference, handle)


In [170]:
with open("density_estimator.pkl", "wb") as handle:
    pickle.dump(density_estimator, handle)

In [171]:
posterior = inference.build_posterior()

In [172]:
with open("posterior.pkl", "wb") as handle:
    pickle.dump(posterior, handle)

In [173]:
samples = posterior.sample((1_000,), x=torch.as_tensor(x_o, dtype=torch.float32))

Drawing 1000 posterior samples for 1 observations: 1034it [00:00, 12325.67it/s]                            


In [174]:
import pandas as pd

p1 = create_prior()
pars = p1.sample((1,))
column_names = pars.columns

parameter_set_pd = pd.DataFrame(np.asarray(samples), columns=column_names)

In [175]:
parameter_set_pd

AB/PD                                                        \
             Na       CaT       CaS          A        KCa          Kd   
0     46.647255  3.313017  3.637501  46.572376   5.354950  148.255371   
1    174.689438  5.962212  3.083156  54.449951   6.779603  124.330956   
2    246.335739  1.999991  2.873050  42.794018   6.876059   46.711960   
3    405.988220  6.150529  3.655866  19.326723  10.135758  122.661797   
4    310.429016  4.208948  0.948971   2.993632   2.324312   26.803862   
..          ...       ...       ...        ...        ...         ...   
995  334.158997  5.729026  4.146793  40.610401   6.097435   43.857319   
996  414.570862  2.985882  7.069912  10.980674  14.764997   86.996483   
997  233.799973  4.846677  1.393343   4.704099   5.432925  118.696136   
998  102.071663  2.643890  2.067126  14.372545   5.990104   80.225098   
999  153.260208  4.128378  5.062061  45.659504  10.309646  149.086929   

                                 LP            ...          PY            \
            H      Leak          Na       CaT  ...          Kd         H   
0    0.018443  0.000258   88.836349  0.912905  ...  131.432358  0.050708   
1    0.010181  0.003383  163.129623  2.466529  ...   67.555367  0.012279   
2    0.013354  0.002054  129.456665  1.140694  ...  146.159988  0.045790   
3    0.008442  0.002208   62.466343  2.036867  ...  144.195679  0.044526   
4    0.001578  0.004754  187.444672  0.980183  ...  137.751083  0.048905   
..        ...       ...         ...       ...  ...         ...       ...   
995  0.002142  0.003546   83.829018  0.951302  ...  100.351219  0.010072   
996  0.004983  0.000103  162.729446  0.658701  ...  137.001160  0.044402   
997  0.008101  0.003789  103.951576  1.821932  ...  144.386536  0.049926   
998  0.007453  0.002564  185.157867  0.169995  ...  143.960373  0.050736   
999  0.007433  0.001289  152.025482  1.644388  ...   78.413307  0.047753   

                Synapses                                              \
         Leak      AB-LP      PD-LP      AB-PY      PD-PY      LP-PD   
0    0.018349 -11.095148 -17.087610 -13.485684  -7.941682 -11.824217   
1    0.005968 -11.467120 -12.168743  -8.773601 -10.367832 -11.056587   
2    0.008614 -11.268309 -13.088203 -10.386112 -14.357227 -10.094992   
3    0.010615 -15.741422 -10.666161  -9.104174  -8.907227  -9.335066   
4    0.023874 -12.671440 -17.212883  -8.133807 -13.441358  -9.416703   
..        ...        ...        ...        ...        ...        ...   
995  0.036682  -5.732061 -16.524473 -11.740211  -9.405987  -8.873189   
996  0.013919  -7.110206 -14.089757 -14.230254  -7.940901 -11.802006   
997  0.019074 -17.184706 -11.920684 -14.543834 -16.569510 -10.002701   
998  0.034359 -14.375953 -11.471232 -13.462841 -10.005461 -11.971454   
999  0.030341  -9.719998 -18.248720 -10.087615  -8.437264  -8.644279   

                           
         LP-PY      PY-LP  
0   -13.993411 -18.259790  
1   -13.271864 -12.396124  
2   -15.237339 -11.348691  
3   -17.059938 -11.580010  
4   -14.693995  -8.883290  
..         ...        ...  
995 -16.604530 -11.009426  
996 -12.363674  -8.667395  
997 -17.428791 -13.614276  
998 -17.076530 -17.596041  
999 -13.696742  -8.950272  

[1000 rows x 31 columns]

In [176]:
all_stats = []
for i in range(20):
    simulation_output = simulate(parameter_set_pd.loc[i])
    summary_statistics = summary_stats(simulation_output)
    all_stats.append(summary_statistics)
summary_statistics = pd.concat(all_stats)

/Users/michaeldeistler/Documents/phd/pyloric/pyloric/utils/circuit_parameters.py:57: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  default_neurons_pd.loc[0][tn, cn] = circuit_parameters[tn, cn] * 0.628e-3
/Users/michaeldeistler/Documents/phd

In [125]:
summary_statistics

cycle_period burst_durations                           duty_cycles  \
       Circuit              PM           LP           PY          PM   
0          NaN      376.000000    90.402273   289.215625         NaN   
0  1317.282143      290.440625   209.877778   350.671875    0.220485   
0  1130.987500      244.547222   317.336111   271.231250    0.216225   
0  1377.782143      530.700000    70.856250   325.018750    0.385184   
0  1139.012500      340.591667    43.170000   206.925000    0.299024   
0  1020.588889      107.510000   301.485000   321.472500    0.105341   
0  2771.050000      129.993750  1579.405000  1590.630000    0.046911   
0  1022.950000      104.007500   323.471429   469.631250    0.101674   
0  1540.545833      192.339286   402.016667   252.046875    0.124851   
0          NaN             NaN  9940.200000   309.987500         NaN   
0  1118.087500      241.163889   205.436111   288.772222    0.215693   
0  1011.013889      156.357500   166.262500   302.559091    0.154654   
0  1656.880000      243.583333   242.800000  1174.500000    0.147013   
0   837.740000      185.106818   254.209375   260.795833    0.220960   
0  1203.846429      313.518750   158.822222   301.867500    0.260431   
0   886.509091      127.481250   177.179167   275.604167    0.143801   
0  1036.611111      139.332500   184.433333   297.480000    0.134412   
0  1062.869444      218.905000   135.875000   273.157500    0.205957   
0          NaN     9963.575000          NaN  9978.650000         NaN   
0          NaN             NaN  9984.025000          NaN         NaN   

                      start_phases           starts_to_starts              \
         LP        PY           LP        PY         AB/PD-LP       LP-PY   
0       NaN       NaN          NaN       NaN              NaN         NaN   
0  0.159326  0.266209          NaN       NaN              NaN         NaN   
0  0.280583  0.239818     0.348379  0.751705       394.012500  456.156250   
0  0.051428  0.235900          NaN       NaN              NaN         NaN   
0  0.037901  0.181671          NaN       NaN              NaN         NaN   
0  0.295403  0.314987     0.310363  0.621703       316.752778  317.750000   
0  0.569966  0.574017     0.186557  0.186488       516.958333   -0.191667   
0  0.316214  0.459095          NaN       NaN              NaN         NaN   
0  0.260957  0.163609          NaN       NaN              NaN         NaN   
0       NaN       NaN          NaN       NaN              NaN         NaN   
0  0.183739  0.258273     0.433838  0.685107       485.068750  280.940625   
0  0.164451  0.299263     0.388919  0.638289       393.202778  252.116667   
0  0.146540  0.708862          NaN       NaN              NaN         NaN   
0  0.303447  0.311309          NaN       NaN              NaN         NaN   
0  0.131929  0.250752          NaN       NaN              NaN         NaN   
0  0.199862  0.310887     0.373504  0.657636       331.115000  251.885000   
0  0.177920  0.286974          NaN       NaN              NaN         NaN   
0  0.127838  0.257000          NaN       NaN              NaN         NaN   
0       NaN       NaN          NaN       NaN              NaN         NaN   
0       NaN       NaN          NaN       NaN              NaN         NaN   

  ends_to_starts              phase_gaps            
        AB/PD-LP        LP-PY   AB/PD-LP     LP-PY  
0            NaN          NaN        NaN       NaN  
0            NaN          NaN        NaN       NaN  
0     145.059375   123.356250   0.128259  0.109070  
0            NaN          NaN        NaN       NaN  
0            NaN          NaN        NaN       NaN  
0     205.350000    16.683333   0.201207  0.016347  
0     376.566667 -2238.483333   0.135893 -0.807811  
0            NaN          NaN        NaN       NaN  
0            NaN          NaN        NaN       NaN  
0            NaN          NaN        NaN       NaN  
0     235.656250    76.906250   0.210767  0.068784  
0     244.327778    89.666667   0

In [136]:
summary_statistics

cycle_period burst_durations                           duty_cycles  \
       Circuit              PM           LP           PY          PM   
0   973.394444      189.017500   135.850000  4412.300000    0.194184   
0  1046.063889      187.775000   182.675000   239.010000    0.179506   
0  1436.870833      166.028571   225.475000   563.285714    0.115549   
0  1247.070833      251.771429   273.293750   478.047222    0.201890   
0          NaN             NaN          NaN  9964.050000         NaN   
0  1001.086111      208.590000   405.579545   204.000000    0.208364   
0  1180.046875      244.883333   248.368750   540.340625    0.207520   
0  1187.256250      205.363889   131.325000   524.783333    0.172974   
0  1076.416667      200.267500   218.980556   320.325000    0.186050   
0  1530.670000      237.616667   646.142857   769.992857    0.155237   
0  2780.458333      239.293750   232.704167  1396.750000    0.086063   
0  1026.475000      251.369444   216.262500   145.872500    0.244886   
0   987.150000      111.355000   156.877500   354.707500    0.112805   
0  1047.512500      220.366667   166.240000   378.802500    0.210371   
0  1371.710714      343.975000   156.478571   525.885714    0.250764   
0          NaN      175.575000  9972.200000  4676.775000         NaN   
0  1363.658333      304.942857   319.206250   555.956250    0.223621   
0   992.369444      184.122500   384.662500  2845.433333    0.185538   
0   928.602500      166.200000   178.534091   162.937500    0.178979   
0  1534.616667      215.317857   252.421429   685.800000    0.140307   

                      start_phases           starts_to_starts              \
         LP        PY           LP        PY         AB/PD-LP       LP-PY   
0  0.139563  4.532900          NaN       NaN              NaN         NaN   
0  0.174631  0.228485          NaN       NaN              NaN         NaN   
0  0.156921  0.392023     0.357788  0.589722       514.095833  333.258333   
0  0.219149  0.383336     0.339776  0.563237       423.725000  278.670833   
0       NaN       NaN          NaN       NaN              NaN         NaN   
0  0.405140  0.203779          NaN       NaN              NaN         NaN   
0  0.210474  0.457898     0.459992  0.533787       542.812500   87.081250   
0  0.110612  0.442014          NaN       NaN              NaN         NaN   
0  0.203435  0.297585          NaN       NaN              NaN         NaN   
0  0.422131  0.503043     0.374310  0.474116       572.945000  152.770000   
0  0.083693  0.502345          NaN       NaN              NaN         NaN   
0  0.210685  0.142110     0.541364  0.638465       555.696875   99.671875   
0  0.158920  0.359325          NaN       NaN              NaN         NaN   
0  0.158700  0.361621          NaN       NaN              NaN         NaN   
0  0.114075  0.383379     0.436949  0.607667       599.367857  234.175000   
0       NaN       NaN          NaN       NaN              NaN         NaN   
0  0.234081  0.407695     0.472265  0.551155       644.008333  107.579167   
0  0.387620  2.867313          NaN       NaN              NaN         NaN   
0  0.192261  0.175465     0.307917  0.801481       285.932500  458.325000   
0  0.164485  0.446887     0.310368  0.545761       476.295833  361.237500   

  ends_to_starts             phase_gaps            
        AB/PD-LP       LP-PY   AB/PD-LP     LP-PY  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0     347.154167  112.887500   0.241604  0.078565  
0     166.795833    7.970833   0.133750  0.006392  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0     299.350000 -161.287500   0.253676 -0.136679  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0     338.550000 -505.970000   0.221178 -0.330555  
0            NaN         NaN        NaN       NaN  
0     299.203125 -117.903125   0.291486 -0.114

In [147]:
summary_statistics

cycle_period burst_durations                          duty_cycles            \
       Circuit              PM          LP           PY          PM        LP   
0  1217.653571      135.681250         NaN   537.136111    0.111428       NaN   
0  1396.966667      324.896429  239.453571   431.328125    0.232573  0.171410   
0  1183.115625      221.383333  186.490625   427.153125    0.187119  0.157627   
0  1074.315625      184.097222  244.777500   344.091667    0.171362  0.227845   
0  1236.564286      210.584375  263.330556   321.187500    0.170298  0.212953   
0  1215.217857      147.521875  252.383333   540.346875    0.121395  0.207686   
0   911.355000      113.815909  187.530556   263.778125    0.124886  0.205771   
0  1144.840625      212.269444  163.561111   475.297222    0.185414  0.142868   
0  1141.543750      309.725000  172.377778   318.136111    0.271321  0.151004   
0  1126.637500      198.611111  221.386111   454.947222    0.176287  0.196502   
0  1424.716667      222.646429  426.764286   335.206250    0.156274  0.299543   
0  1153.765625      248.536111  208.100000   430.161111    0.215413  0.180366   
0  1143.212500      239.088889  315.808333   298.772222    0.209138  0.276246   
0   972.908333      180.940000  184.431818   413.872727    0.185978  0.189568   
0  1095.890625      234.194444  176.152778   262.177500    0.213702  0.160739   
0          NaN       87.125000  103.102083  4736.637500         NaN       NaN   
0   983.666667      187.965000  173.497222   189.665909    0.191086  0.176378   
0  1098.478125      175.255556  211.533333   310.736111    0.159544  0.192569   
0  1140.378125      173.572222  227.902778   383.755556    0.152206  0.199848   
0  1091.081250      219.061111  227.027778   333.330556    0.200774  0.208076   

            start_phases           starts_to_starts              \
         PY           LP        PY         AB/PD-LP       LP-PY   
0  0.441124          NaN       NaN              NaN         NaN   
0  0.308760     0.412668  0.597348       576.483333  257.991667   
0  0.361041          NaN       NaN              NaN         NaN   
0  0.320289     0.397686  0.657776       427.240625  279.418750   
0  0.259742     0.391170  0.742561       483.707143  434.517857   
0  0.444650     0.359539  0.569225       436.917857  254.814286   
0  0.289435          NaN       NaN              NaN         NaN   
0  0.415165     0.494578  0.538983       566.212500   50.837500   
0  0.278689     0.485431  0.664818       554.140625  204.778125   
0  0.403810     0.383969  0.591814       432.593750  234.165625   
0  0.235279     0.364774  0.696416       519.700000  472.495833   
0  0.372832          NaN       NaN              NaN         NaN   
0  0.261344     0.216143  0.674106       247.096875  523.550000   
0  0.425397     0.465260  0.549444       452.655556   81.902778   
0  0.239237     0.442217  0.738438       484.621875  324.625000   
0       NaN          NaN       NaN              NaN         NaN   
0  0.192815          NaN       NaN              NaN         NaN   
0  0.282879     0.444678  0.714705       488.468750  296.618750   
0  0.336516     0.440449  0.623579       502.278125  208.837500   
0  0.305505     0.446810  0.681265       487.506250  255.809375   

  ends_to_starts             phase_gaps            
        AB/PD-LP       LP-PY   AB/PD-LP     LP-PY  
0            NaN         NaN        NaN       NaN  
0     253.654167   18.708333   0.181575  0.013392  
0            NaN         NaN        NaN       NaN  
0     244.915625   26.362500   0.227974  0.024539  
0     268.564286  153.432143   0.217186  0.124079  
0     285.503571  -16.207143   0.234940 -0.013337  
0            NaN         NaN        NaN       NaN  
0     351.753125 -127.418750   0.307251 -0.111298  
0     246.968750   28.721875   0.216346  0.025161  
0     225.721875   18.331250   0.200350  0.016271  
0     297.387500   49.458333   0.208734  0.034715  
0            NaN         NaN        NaN       NaN  
0      -1.359375

In [157]:
summary_statistics

cycle_period burst_durations                           duty_cycles  \
       Circuit              PM           LP           PY          PM   
0  1546.258333      220.082143   207.946429   716.789286    0.142332   
0  1086.211111      179.760000   314.958333   458.887500    0.165493   
0  1009.822222      190.182500   245.825000   285.559091    0.188333   
0  1129.068750      189.705556   160.350000   430.461111    0.168019   
0  1063.809375      163.480556   158.120833   562.150000    0.153675   
0  1086.643750      265.630556   234.913889   224.350000    0.244450   
0  1062.200000      198.262500   222.425000   365.727500    0.186653   
0  1092.737500      226.402778   173.435000   291.272222    0.207189   
0  1139.850000      210.755556   206.836111   375.717500    0.184898   
0   860.690909      173.095833          NaN  9973.000000    0.201113   
0  1163.762500      409.933333   115.075000   404.275000    0.352248   
0          NaN      230.575000   102.231250  4595.337500         NaN   
0  1097.400000      222.636111          NaN  9934.700000    0.202876   
0  1094.290625      179.872222   173.871429   456.686111    0.164373   
0  1086.628125      201.347222   192.208333   495.225000    0.185295   
0  1059.647222      150.560000          NaN   366.694444    0.142085   
0  1189.465625      173.377778   152.612500   459.322222    0.145761   
0  1141.434375      242.741667   178.931250   279.913889    0.212664   
0          NaN             NaN   102.067857  9985.925000         NaN   
0  2085.687500      244.025000  1396.700000   298.733333    0.117000   

                       start_phases           starts_to_starts              \
         LP         PY           LP        PY         AB/PD-LP       LP-PY   
0  0.134484   0.463564          NaN       NaN              NaN         NaN   
0  0.289961   0.422466     0.450542  0.532457       489.383333   88.977778   
0  0.243434   0.282782          NaN       NaN              NaN         NaN   
0  0.142020   0.381253     0.457266  0.586382       516.284375  145.781250   
0  0.148636   0.528431          NaN       NaN              NaN         NaN   
0  0.216183   0.206461     0.504783  0.779289       548.518750  298.290625   
0  0.209400   0.344311     0.425349  0.618854       451.805556  205.541667   
0  0.158716   0.266553     0.488784  0.715691       534.112500  247.950000   
0  0.181459   0.329620     0.418087  0.604773       476.556250  212.793750   
0       NaN  11.587203          NaN       NaN              NaN         NaN   
0  0.098882   0.347386          NaN       NaN              NaN         NaN   
0       NaN        NaN          NaN       NaN              NaN         NaN   
0       NaN   9.052943          NaN       NaN              NaN         NaN   
0  0.158890   0.417335          NaN       NaN              NaN         NaN   
0  0.176885   0.455745          NaN       NaN              NaN         NaN   
0       NaN   0.346053          NaN       NaN              NaN         NaN   
0  0.128303   0.386158     0.456720  0.599899       543.253125  170.306250   
0  0.156760   0.245230     0.476402  0.754779       543.781250  317.750000   
0       NaN        NaN          NaN       NaN              NaN         NaN   
0  0.669659   0.143230          NaN       NaN              NaN         NaN   

  ends_to_starts             phase_gaps            
        AB/PD-LP       LP-PY   AB/PD-LP     LP-PY  
0            NaN         NaN        NaN       NaN  
0     295.280556 -225.980556   0.271845 -0.208045  
0            NaN         NaN        NaN       NaN  
0     330.828125  -21.487500   0.293010 -0.019031  
0            NaN         NaN        NaN       NaN  
0     284.253125   62.728125   0.261588  0.057726  
0     251.352778  -16.883333   0.236634 -0.015895  
0     300.940625   76.725000   0.275401  0.070214  
0     257.550000    8.059375   0.225951  0.007071  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0            NaN        

In [167]:
summary_statistics

cycle_period burst_durations                           duty_cycles  \
       Circuit              PM           LP           PY          PM   
0  1090.459375      115.586111   151.212500   408.602778    0.105998   
0   978.308333      114.212500   209.556818   362.170000    0.116745   
0  1761.595000      132.670833   427.346875   661.300000    0.075313   
0  1234.657143      183.953125   321.403125   223.228571    0.148991   
0  1143.703125      183.036111   133.096429   527.041667    0.160038   
0  1081.346875      179.805556   128.830000   279.890000    0.166279   
0  1222.825000      168.096875   205.175000   465.461111    0.137466   
0  1376.491667      192.389286   335.089286   267.760714    0.139768   
0  1220.093750      241.972222   247.868750   394.315625    0.198323   
0          NaN             NaN   157.347917   581.528571         NaN   
0          NaN             NaN  9962.925000  9954.900000         NaN   
0  1100.305556      167.850000   249.380556   394.516667    0.152549   
0   995.163889      177.165000   126.290000   276.847727    0.178026   
0  1029.291667      143.680000   158.715000   371.545000    0.139591   
0  1075.213889      188.905000   232.913889   272.719444    0.175691   
0  1141.025000      171.011111   182.055556   450.666667    0.149875   
0  1153.807143      255.971875   328.277778   299.625000    0.221850   
0  1108.600000      207.527778   110.341667   290.725000    0.187198   
0  1228.278571      130.262500   413.483333   520.453125    0.106053   
0  1418.633333      503.578571   305.828571   514.721429    0.354974   

                      start_phases           starts_to_starts              \
         LP        PY           LP        PY         AB/PD-LP       LP-PY   
0  0.138669  0.374707     0.331360  0.606648       361.334375  300.190625   
0  0.214203  0.370200     0.301970  0.620002       295.419444  311.133333   
0  0.242591  0.375398          NaN       NaN              NaN         NaN   
0  0.260318  0.180802          NaN       NaN              NaN         NaN   
0  0.116373  0.460820          NaN       NaN              NaN         NaN   
0  0.119138  0.258835     0.495710  0.722325       536.034375  245.050000   
0  0.167788  0.380644     0.404990  0.575406       495.232143  208.389286   
0  0.243437  0.194524          NaN       NaN              NaN         NaN   
0  0.203155  0.323185     0.384658  0.663661       469.318750  340.409375   
0       NaN       NaN          NaN       NaN              NaN         NaN   
0       NaN       NaN          NaN       NaN              NaN         NaN   
0  0.226647  0.358552     0.367991  0.632193       404.902778  290.702778   
0  0.126904  0.278193     0.479031  0.643914       476.713889  164.086111   
0  0.154198  0.360972     0.430528  0.598278       443.138889  172.663889   
0  0.216621  0.253642          NaN       NaN              NaN         NaN   
0  0.159554  0.394967     0.402040  0.568628       458.737500  190.081250   
0  0.284517  0.259684     0.300824  0.722075       347.092857  486.042857   
0  0.099532  0.262245     0.448533  0.725930       497.243750  307.521875   
0  0.336636  0.423726          NaN       NaN              NaN         NaN   
0  0.215580  0.362829     0.523726  0.630281       742.975000  151.162500   

  ends_to_starts             phase_gaps            
        AB/PD-LP       LP-PY   AB/PD-LP     LP-PY  
0     244.071875  141.528125   0.223825  0.129788  
0     182.580556   91.661111   0.186629  0.093693  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0     355.625000  106.971875   0.328872  0.098925  
0     338.360714  -12.989286   0.276704 -0.010622  
0            NaN         NaN        NaN       NaN  
0     208.203125   92.540625   0.170645  0.075847  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0     225.969444   41.322222   0.205370  0.037

In [177]:
summary_statistics

cycle_period burst_durations                         duty_cycles            \
       Circuit              PM          LP          PY          PM        LP   
0          NaN       43.175000  187.755000  293.725000         NaN       NaN   
0  1286.378571      190.587500  439.181250  305.143750    0.148158  0.341409   
0  1900.765000      155.562500  161.215909  162.506250    0.081842  0.084816   
0  1683.537500      216.365000  141.495833  316.436111    0.128518  0.084047   
0  2423.850000       95.070000  261.360000  431.907500    0.039223  0.107828   
0  1191.768750      208.533333  253.721875  484.763889    0.174978  0.212895   
0  1184.721429      212.409375  222.263889  504.056250    0.179291  0.187609   
0  7867.700000      964.175000  236.432143  630.764286    0.122549  0.030051   
0  1038.843750      124.338889  240.437500  294.465000    0.119690  0.231447   
0  1361.195833      212.117857  383.717857  591.581250    0.155832  0.281898   
0  1571.955000      224.520833  359.218750  468.621875    0.142829  0.228517   
0  1245.900000      122.428571   71.475000  427.203125    0.098265  0.057368   
0  1499.716667      186.835714  137.068182  425.106250    0.124581  0.091396   
0  1325.975000      199.521429  393.653125  552.484375    0.150471  0.296878   
0  2006.793750      408.020000         NaN  145.275000    0.203319       NaN   
0  1266.592857      207.109375  347.046875  522.318750    0.163517  0.274000   
0  1217.814286      173.818750  262.865000  479.059375    0.142730  0.215850   
0  1140.878125      181.133333  163.240625  354.311111    0.158767  0.143083   
0  1017.519444      155.400000  264.232500  416.277273    0.152724  0.259683   
0  1132.743750      220.622222  171.872222  388.990000    0.194768  0.151731   

            start_phases           starts_to_starts              \
         PY           LP        PY         AB/PD-LP       LP-PY   
0       NaN          NaN       NaN              NaN         NaN   
0  0.237211     0.348820  0.719581       448.714286  476.939286   
0  0.085495          NaN       NaN              NaN         NaN   
0  0.187959          NaN       NaN              NaN         NaN   
0  0.178191          NaN       NaN              NaN         NaN   
0  0.406760     0.254349  0.539379       303.125000  339.690625   
0  0.425464     0.276575  0.572172       327.664286  350.200000   
0  0.080171          NaN       NaN              NaN         NaN   
0  0.283455     0.256574  0.686042       266.540625  446.150000   
0  0.434604     0.380780  0.511756       518.316667  178.283333   
0  0.298114          NaN       NaN              NaN         NaN   
0  0.342887          NaN       NaN              NaN         NaN   
0  0.283458          NaN       NaN              NaN         NaN   
0  0.416663     0.382706  0.497678       507.458333  152.450000   
0  0.072392          NaN       NaN              NaN         NaN   
0  0.412381     0.251189  0.570284       318.153571  404.164286   
0  0.393376          NaN       NaN              NaN         NaN   
0  0.310560     0.441486  0.627976       503.681250  212.762500   
0  0.409110     0.338435  0.533140       344.363889  198.116667   
0  0.343405     0.483781  0.579114       548.000000  107.987500   

  ends_to_starts             phase_gaps            
        AB/PD-LP       LP-PY   AB/PD-LP     LP-PY  
0            NaN         NaN        NaN       NaN  
0     257.235714   23.492857   0.199969  0.018263  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0      97.271875   85.968750   0.081620  0.072135  
0     120.385714  134.692857   0.101615  0.113692  
0            NaN         NaN        NaN       NaN  
0     142.493750  202.550000   0.137166  0.194976  
0     309.075000 -210.954167   0.227061 -0.154977  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        NaN       NaN  
0            NaN         NaN        Na